In [4]:
!pip install alpaca-py --trusted-host pypi.org --trusted-host files.pythonhosted.org
 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 8.4 MB/s eta 0:00:00
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.7/121.7 kB 1.6 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.0/430.0 kB 8.5 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 1.4 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 3.6 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 1.9 MB/s eta 0:00:00a 0:00:01m
Using cached ann

In [19]:
!pip install Path  --trusted-host pypi.org --trusted-host files.pythonhosted.org

In [1]:
import pandas as pd
import os
from pathlib import Path
import sys

import matplotlibas pl

In [2]:
PATH_PROJECT_ROOT_DIR = Path(os.getcwd()).resolve().parent
PATH_SRC_DIR = os.path.join(PATH_PROJECT_ROOT_DIR, "src/")

sys.path.append(str(PATH_SRC_DIR))


In [5]:
input_tw

,tweet,sentiment,url
0,$BYND - JPMorgan reels in expectations on Beyo...,2,https://huggingface.co/datasets/zeroshot/twitt...
1,$CCL $RCL - Nomura points to bookings weakness...,2,https://huggingface.co/datasets/zeroshot/twitt...
2,"$CX - Cemex cut at Credit Suisse, J.P. Morgan ...",2,https://huggingface.co/datasets/zeroshot/twitt...
3,$ESS: BTIG Research cuts to Neutral https://t....,2,https://huggingface.co/datasets/zeroshot/twitt...
4,$FNKO - Funko slides after Piper Jaffray PT cu...,2,https://huggingface.co/datasets/zeroshot/twitt...
...,...,...,...
38086,Facebook $FB received a Buy rating from Wells ...,1,https://huggingface.co/datasets/ChanceFocus/fi...
38087,$TSLA Wish had my puts back but see if we can ...,2,https://huggingface.co/datasets/ChanceFocus/fi...
38088,Citrix Systems Inc $CTXS Position Increased by...,1,https://huggingface.co/datasets/ChanceFocus/fi...
38089,Notable gainers among liquid option names this...,1,https://huggingface.co/datasets/ChanceFocus/fi...


In [4]:
input_tw = pd.read_parquet("data/train-00000-of-00001.parquet")

# Quick EDA
print("Shape:", input_tw.shape)
print("\nColumns:", input_tw.columns.tolist())
print("\nMissing values:\n", input_tw.isnull().sum())
print("\nSentiment value counts:\n", input_tw['sentiment'].value_counts())
print("\nTweet length stats:\n", input_tw['tweet'].apply(len).describe())

Shape: (38091, 3)

Columns: ['tweet', 'sentiment', 'url']

Missing values:
 tweet        0
sentiment    0
url          0
dtype: int64

Sentiment value counts:
 sentiment
1    17368
0    12181
2     8542
Name: count, dtype: int64

Tweet length stats:
 count    38091.000000
mean        95.323751
std         72.693280
min          1.000000
25%         55.000000
50%         84.000000
75%        131.000000
max       4229.000000
Name: tweet, dtype: float64


In [6]:
input_tw.head(5)

,tweet,sentiment,url
0,$BYND - JPMorgan reels in expectations on Beyo...,2,https://huggingface.co/datasets/zeroshot/twitt...
1,$CCL $RCL - Nomura points to bookings weakness...,2,https://huggingface.co/datasets/zeroshot/twitt...
2,"$CX - Cemex cut at Credit Suisse, J.P. Morgan ...",2,https://huggingface.co/datasets/zeroshot/twitt...
3,$ESS: BTIG Research cuts to Neutral https://t....,2,https://huggingface.co/datasets/zeroshot/twitt...
4,$FNKO - Funko slides after Piper Jaffray PT cu...,2,https://huggingface.co/datasets/zeroshot/twitt...


In [ ]:
text_lengths = input_tw['tweet'].str.split().str.len()

# Calculate the average
average_length = np.mean(text_lengths)

# Create a figure and axis
fig, ax = plt.subplots(figsize=(12, 6))

# Using plt.hist to create a histogram with Matplotlib
ax.hist(text_lengths, bins=20, color="blue", edgecolor="black", alpha=0.7)

# Add average line
ax.axvline(average_length, color='red', linestyle='dashed', linewidth=2, label=f'Average: {average_length:.2f}')

ax.set_title('Histogram of Tweet Lengths')
ax.set_xlabel('Tweet Length')
ax.set_ylabel('Count')

# Display the plot
plt.show()